In [21]:
import sys
sys.path.append('../60_Code/generate_patterns/')
sys.path.append('../30_data_tools/')

from raster_image_processor import RasterImageProcessor, DOT_SHAPE
from PIL import Image, ImageFont, ImageDraw  
from helper import load_dotenv
from pathlib import Path
import numpy as np

In [4]:
dotenv = load_dotenv()

In [ ]:
rip = RasterImageProcessor( DOT_SHAPE.DIAMOND, 16 )

In [ ]:
coverage = .3 * 255

In [ ]:
img = Image.new(
    "L",
    (1600,1600),
    color='white'
)

draw = ImageDraw.Draw(img)
font = ImageFont.load_default(size=25)

for i in range(100, img.size[0], 100):
    draw.line((i, 0, i, img.size[1]), fill=128)

for i in range(100, img.size[1], 100):
    draw.line((0, i, img.size[0]+1, i), fill=128)


for y in range(rip.threshold_map.shape[0]):
    for x in range(rip.threshold_map.shape[1]):
        if rip.threshold_map[y,x] <= coverage:
            color = 'white'
            draw.rectangle(
                (x*100, y*100, (x+1)*100, (y+1)*100),
                fill='black'
            )
        else:
            color = 'black'
        
        draw.text(
            (x*100 + 50, y*100 + 50),
            str(int(rip.threshold_map[y,x])),
            font=font,
            fill=color,
            anchor="mm"
        )

In [ ]:
img.save( dotenv['ATTACHMENT_DIR'] / 'Rasterzelle_30P.jpg')

# Schwellwertmatrizen

In [26]:
import csv
import plotly.express as px

In [35]:
attachment_dir = dotenv['ATTACHMENT_DIR'] / 'schwellwertmatrizen'

In [39]:
def get_threshold_matrix( csv_path ):
        
    with csv_path.open() as csv_file:
        reader = csv.reader( csv_file )

        values = []
        for row in reader:
            values.append([int(float(v)) for v in row[0].split(';')])

    return np.array(values)

def get_threshold_fig( csv_path ):
    fig = px.imshow(
        get_threshold_matrix( csv_path ),
        text_auto=True,
        color_continuous_scale=['white','black'],
        width=1000,
        height=1000
    )
    fig.update_layout(coloraxis_showscale=False)
    fig.update_xaxes(showticklabels=False)
    fig.update_yaxes(showticklabels=False)

    return fig

In [40]:
line_path = Path('../60_Code/generate_patterns/line_16.csv')
circle_path = Path('../60_Code/generate_patterns/circle_16.csv')
diamond_path = Path('../60_Code/generate_patterns/diamond_16.csv')

In [41]:
get_threshold_fig( line_path ).write_image( attachment_dir / 'line_16.pdf' )
get_threshold_fig( circle_path ).write_image( attachment_dir / 'circle_16.pdf' )
get_threshold_fig( diamond_path ).write_image( attachment_dir / 'diamond_16.pdf' )